In [1]:
#this scripts takes the output from var_builder
#converts timedelta to numeric
#calculates backward citation average delay
#calculates forward citation average delay
#counts forward and backward citations

# Jan 19, 2020
# Joining is not generating matches
# probably is an issue with dtype
# i am running joining with inner but it should be updated later with outer.

In [2]:
import sys
sys.path.append('/home/rkogeyam/scripts/')
from sampler import sampler

import pandas as pd
import numpy as np
import re

In [3]:
src='data/var_builder.csv'
dst='data/cit_delay.csv'
cit_tree = 'data/cit_tree.csv'

In [4]:
src='data/var_builder.csv'
dst='data/var_builder2.csv'
cit_tree = 'data/cit_tree.csv'

cols=['patent_id','citation_id', 'cit_delay' ]

df = pd.read_csv(src, usecols=cols, converters={'citation_id':str})

In [5]:
df['cit_delay']=pd.to_timedelta(df['cit_delay']).dt.components.days/365 
#for some reason dt.days stopped working. dt.components came around and is working

In [6]:
# output: patent-level data
# if group by citation_id, the delay will give the average delay gives how much time it took,
#    in average, for a patent to receive citations
# it is a measure of time to become influent

back_delay=df.groupby('citation_id').cit_delay.agg(['mean', 'count'])
back_delay.rename(columns={'count': 'back_citation', 'mean':'mean_back_delay'}, inplace=True)
back_delay.head()

,mean_back_delay,back_citation
citation_id,,
,NaN,0
0,NaN,0
0000000,16.443836,1
000004,169.767123,1
000006,168.367123,1


In [7]:
back_delay.dtypes #this will tell us if i am trying to join data of the same format

mean_back_delay    float64
back_citation        int64
dtype: object

In [8]:
# if you group by patent_id, the delay will give a measure of how far back the patent is rooted.

forw_delay=df.groupby('patent_id').cit_delay.agg(['mean', 'count'])
forw_delay.rename(columns={'count': 'forw_citation', 'mean':'mean_forw_delay'}, inplace=True)
forw_delay.head()

,mean_forw_delay,forw_citation
patent_id,,
3930271,24.738356,2
3930272,34.159402,11
3930273,16.671575,8
3930274,14.164384,3
3930275,11.854795,2


In [9]:
forw_delay.dtypes

mean_forw_delay    float64
forw_citation        int64
dtype: object

In [10]:
output=forw_delay.join(back_delay, how='outer')

In [11]:
output.describe()

/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/numpy/lib/function_base.py:4269: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


,mean_forw_delay,forw_citation,mean_back_delay,back_citation
count,5.972148e+06,6.188752e+06,7.020620e+06,7.051647e+06
mean,1.369108e+01,1.438777e+01,2.039864e+01,1.262717e+01
std,1.066943e+01,4.185029e+01,2.430667e+01,2.997653e+01
min,-2.805233e+02,0.000000e+00,-3.514364e+01,0.000000e+00
25%,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN
max,1.827425e+02,5.811000e+03,2.075753e+02,3.725000e+03


In [12]:
output.head()

,mean_forw_delay,forw_citation,mean_back_delay,back_citation
,NaN,NaN,NaN,0.0
0,NaN,NaN,NaN,0.0
0000000,NaN,NaN,16.443836,1.0
000004,NaN,NaN,169.767123,1.0
000006,NaN,NaN,168.367123,1.0


In [23]:
output.to_csv(dst)